# 1.探索性分析所有顧客資料的特徵(視覺化呈現)，並處理資料的格式與問題(例如: 遺失值等)。

In [ ]:
!pip install facets-overview

In [ ]:
from facets_overview.generic_feature_statistics_generator import GenericFeatureStatisticsGenerator
from IPython.core.display import display, HTML
import base64

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

sns.set(style="whitegrid")
pd.set_option('display.max_columns', None)
# pd.set_option('display.width',1000)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
# plt.rc('font', family='Microsoft JhengHei') # 微軟正黑體
plt.rcParams['axes.unicode_minus'] = False # 正常顯示負號

In [ ]:
# Colab 進行matplotlib繪圖時顯示繁體中文
# 下載台北思源黑體並命名taipei_sans_tc_beta.ttf，移至指定路徑
!wget -O TaipeiSansTCBeta-Regular.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib as mpl
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

# Exploratory Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_utf8.csv')

df

In [ ]:
# 生成統計信息的 proto
proto_combined = GenericFeatureStatisticsGenerator().ProtoFromDataFrames([{'name': 'Combined Data', 'table': df}])
protostr_combined = base64.b64encode(proto_combined.SerializeToString()).decode("utf-8")

# 創建 HTML 内容
html_content_combined = f'''
    <script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
    <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/master/facets-dist/facets-jupyter.html">
    <facets-overview id="elem_combined"></facets-overview>
    <script>document.querySelector("#elem_combined").protoInput = "{protostr_combined}"</script>
'''
# 顯示 Facets Dive
display(HTML(html_content_combined))

In [ ]:
# 列出欄位唯一值
# for column in df.columns:
#     print(f"Unique values in {column}: {df[column].unique()}")

# Missing Value

平均長途話費、多線路服務：9.68%

平均下載量( GB)、網路連線類型、線上安全服務、線上備份服務、設備保護計劃、技術支援計劃、電視節目、電影節目、音樂節目、無限資料下載：21.67%

客戶流失類別、客戶離開原因：73.46%

存在缺失值

In [ ]:
msno.bar(df)

In [ ]:
msno.matrix(df)

In [ ]:
msno.matrix(df.sample(100))

In [ ]:
categorical_columns = ['網路服務' ,'平均下載量( GB)', '網路連線類型', '線上安全服務', '線上備份服務', '設備保護計劃',
            '技術支援計劃', '電視節目', '電影節目', '音樂節目', '無限資料下載' ,'客戶流失類別' ,'客戶離開原因']

for column in categorical_columns:
    print(f"{column} 中的唯一值: {df[column].unique()}")

In [ ]:
no_network_service_df = df[df['網路服務'] == 'No']

no_network_service_df

經圖表觀察發現，缺失值屬於對稱

可得知平均長途話費為空值時多線路服務也是空值

網路服務為No時平均下載量( GB)、網路連線類型、網路連線類型、線上安全服務、線上備份服務、設備保護計劃、技術支援計劃、電視節目、電影節目、音樂節目、無限資料下載都為空值
客戶流失類別為空值時客戶離開原因也是空值

In [ ]:
msno.heatmap(df)

In [ ]:
msno.dendrogram(df)

In [ ]:
# df = df.dropna()

# df # 1586 rows × 38 columns，暴力刪除後資料筆數過少，因此不採用

In [ ]:
df.fillna('No', inplace=True)

network_service_counts = df['網路服務'].value_counts()
print(network_service_counts)
print(df.info())

# Outliers
每月費用有<0的數值

In [ ]:
# 每月費用、總費用、總退款    這三個一起看
# vars = ['每月費用', '總費用', '總退款']  # 先觀察圖形，找出特別明顯的異常值，再進一步處理
# numeric_df = df[vars]
# sns.pairplot(numeric_df, markers='o')
# plt.show()

negative_monthly_costs = df[df['每月費用'] < 0]
print(negative_monthly_costs) # 列出異常值 (120 ,38)
df = df[df['每月費用'] > 0] # 刪除異常值，只保留正常值

# Processing Data(依照簡報要求)

In [ ]:
def fillna_with_no(df, columns):
    for column in columns:
        df[column] = df[column].fillna('No')
    return df
use_columns = ['線上安全服務', '線上備份服務', '設備保護計劃', '技術支援計劃', '電視節目', '電影節目', '音樂節目', '無限資料下載', '多線路服務']
df = fillna_with_no(df, use_columns)
df

In [ ]:
def fillna_with_value(df, column_value_map):
    for column, value in column_value_map.items():
        df[column] = df[column].fillna(value)
    return df
column_value_map = {
    '優惠方式': 'No Discount',
    '網路連線類型': 'No Type',
    '平均下載量( GB)': 0,
    '平均長途話費': 0,
    '客戶流失類別': 'None',
    '客戶離開原因': 'None'
}
df = fillna_with_value(df, column_value_map)
df

# Output Handled Data

In [ ]:
df.to_csv('customer_data_handled.csv')

df = pd.read_csv('/content/drive/MyDrive/大三/上學期/大數據決策/期末報告/customer_data_handled.csv')

df # (6923, 38)

# Plot Data

In [ ]:
# 年齡
age_counts = df['年齡'].value_counts()
print("年齡的個數:")
print(age_counts)

# 分組為不同年齡段
age_bins = [20, 30, 40, 50, 60, 70, 80, 90]
age_labels = ['19-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

# 將年齡資料分組
df['年齡分組'] = pd.cut(df['年齡'], bins=age_bins, labels=age_labels, right=False)

# # 計算每個年齡段的人數
age_group_counts = df['年齡分組'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
sns.barplot(x=age_group_counts.index, y=age_group_counts.values, palette='viridis')
plt.title('年齡分組計數圖')
plt.xlabel('年齡段')
plt.ylabel('計數')
plt.show()

In [ ]:
# 扶養人數
support_series = df['扶養人數']
value_counts = support_series.value_counts()
print(value_counts)

plt.figure(figsize=(10, 6))
sns.countplot(x='扶養人數', data=df, palette='muted')
plt.title('扶養人數計數圖')
plt.xlabel('扶養人數')
plt.ylabel('計數')
plt.show()

In [ ]:
def plot_categorical_feature_counts(df, feature):
    # Calculate value counts
    feature_counts = df[feature].value_counts()

    # Print counts for each category
    for category, count in feature_counts.items():
        print(f"{category} 的個數: {count}")

    # Plot count plot
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.countplot(x=feature, data=df, palette='Set2')
    plt.title(f'{feature}計數圖')
    plt.xlabel(feature)
    plt.ylabel('計數')

    # Plot pie chart
    plt.subplot(1, 2, 2)
    colors = ['lightskyblue', 'lightcoral']
    plt.pie(feature_counts, labels=feature_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title(f'{feature}圓餅圖')
    plt.show()

def plot_categorical_features(df, use_columns):
    for feature in use_columns:
        plot_categorical_feature_counts(df, feature)

use_columns = ['性別', '婚姻', '優惠方式', '電話服務', '網路服務', '多線路服務','網路連線類型', '線上安全服務', '線上備份服務', '設備保護計劃',
        '技術支援計劃', '電視節目', '電影節目', '音樂節目', '無限資料下載','合約類型', '無紙化計費', '支付帳單方式']
plot_categorical_features(df, use_columns)

In [ ]:
# 城市
city_counts = df['城市'].value_counts()
print(city_counts)
# 城市名稱太多，好像不好畫圖，故只取前10個城市
top_cities = df['城市'].value_counts().head(10)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_cities.values, y=top_cities.index, palette='Set2')
plt.title('前 10 項城市計數圖')
plt.xlabel('計數')
plt.ylabel('城市')
plt.show()

In [ ]:
# 客戶流失類別、客戶離開原因
df_customer_left = df[['客戶流失類別', '客戶離開原因']]

customer_churn_counts = df['客戶流失類別'].value_counts()
print("客戶流失類別計數:\n", customer_churn_counts)
customer_leave_reason_counts = df['客戶離開原因'].value_counts()
print("\n客戶離開原因計數:\n", customer_leave_reason_counts)

df_customer_left = df[['客戶流失類別', '客戶離開原因']].dropna()
print(df_customer_left.shape) # (1839, 2)

plt.figure(figsize=(15, 8))
sns.countplot(x='客戶離開原因', hue='客戶流失類別', data=df_customer_left)
plt.title('客戶離開原因分佈圖')
plt.xlabel('客戶離開原因' ,fontsize=4)
plt.ylabel('計數')
plt.xticks(rotation=25, ha='right')  # 如果x軸的標籤太長，可以選擇旋轉標籤以避免重疊
plt.show()